In [62]:
import pandas as pd # using panda
import numpy as np # using numpy
import matplotlib.pyplot as plt # using matplotlib
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import accuracy_score
from Visualization import *

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import seaborn as sns
import csv
import json
import os
from pymongo import MongoClient

In [2]:
# with open('matchFilteredJson3.29.2024.json', 'r') as file:
#     config = json.load(file)
# client = MongoClient('mongodb://127.0.0.1:27017/')
# db = client['Project']
# collection = db['Data']



## Converting the data from json to a csv file

In [3]:
# df = pd.read_json("matchFilteredJson3.29.2024.json")
# df.to_csv("your_file.csv")

## Load Data
    

In [4]:
df = pd.read_json("matchFilteredJson3.29.2024.json")
df.to_csv("your_file.csv")

# Load Data
# dData = pd.read_csv("your_file.csv")
# df = pd.DataFrame(dData)


## Create DataFrame

### convert from string to list

In [5]:
def parse_string_to_list(string):
    # Check if the input is None
    if string is None:
        return []

    # Check if the input is a string
    if not isinstance(string, str):
        return string

    try:
        # Attempt to parse the string as JSON
        data = json.loads(string)
        # If successful, return the parsed data
        return data
    except json.JSONDecodeError:
        # If parsing fails, return an empty list
        return []

### Extract the amount according to the minutes

In [6]:
def get_statistic_feature(match, minute, statistic_name, number):
    if statistic_name not in match:
        raise Exception(f"No data about {statistic_name} in match {match['_id']}")
    amount = 0
    # todo check why need this for create_score_balance_target
    if match[statistic_name] is None:
        return amount
    for data in parse_string_to_list(match[statistic_name][number]):
        if minute == data['minute']:
            return data['amount']
        if data['minute'] > minute: 
            return amount
        else:
            amount = data['amount']
    return amount

In [7]:


def get_statistic_feature_list_minutes(minutes, statistic_name):
    
    # Initialize lists to store data for each minute value
    lists = [[] for _ in range(len(minutes))]

    # Iterate over the dataframe
    for i in range(len(df)):
        # Iterate over the minute values
        for j, minute in enumerate(minutes):
            # Get data for the current minute value
            value = get_statistic_feature(df, minute, statistic_name, i)
            # Append to the corresponding list
            lists[j].append(value)
    return lists


### Build DataFrame

In [8]:
def buildDataFrame(list_minutes):
    minutes = list_minutes
    statistics = [
        'attacksLocalteam',
        'possessionLocalteam',
        'possessionVisitorTeam',
        'attacksVisitorTeam',
        'dangerousAttacksLocalteam',
        'dangerousAttacksVisitorTeam',
        'offTargetLocalteam',
        'offTargetVisitorTeam',
        'onTargetLocalteam',
        'onTargetVisitorTeam',
        'cornersLocalteam',
        'cornersVisitorTeam',
        'goalsLocalteam',
        'goalsLocalteam'
    ]

    data = {}

    for statistic in statistics:
        statistic_list = get_statistic_feature_list_minutes(minutes, statistic)
        for i, minute in enumerate(minutes):
            data[f"{statistic}_{minute}"] = statistic_list[i]

    return pd.DataFrame(data)


In [9]:
list_minutes = [10, 20, 30, 40, 50, 60]
new_df = buildDataFrame(list_minutes)

### Show DataFrame

In [10]:
new_df

,attacksLocalteam_10,attacksLocalteam_20,attacksLocalteam_30,attacksLocalteam_40,attacksLocalteam_50,attacksLocalteam_60,possessionLocalteam_10,possessionLocalteam_20,possessionLocalteam_30,possessionLocalteam_40,...,cornersVisitorTeam_30,cornersVisitorTeam_40,cornersVisitorTeam_50,cornersVisitorTeam_60,goalsLocalteam_10,goalsLocalteam_20,goalsLocalteam_30,goalsLocalteam_40,goalsLocalteam_50,goalsLocalteam_60
0,20,39,61,78,101,120,66,68,71,71,...,0,0,0,0,0,0,0,0,1,1
1,16,26,33,47,58,70,59,52,48,46,...,5,5,5,5,0,0,0,0,0,0
2,16,24,38,48,62,78,40,43,44,41,...,2,2,2,3,1,1,1,1,1,1
3,21,38,52,64,80,98,51,54,55,55,...,2,2,3,3,0,0,1,1,1,1
4,15,30,34,46,58,70,61,69,72,70,...,0,0,0,0,0,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76385,18,28,43,51,65,76,62,52,52,51,...,0,1,1,2,0,0,0,0,0,0
76386,15,27,29,34,45,59,34,44,41,40,...,2,2,4,4,0,0,0,0,0,0
76387,19,34,49,65,77,89,75,58,53,56,...,0,0,0,0,0,0,0,0,0,0
76388,6,18,31,45,64,77,24,32,36,38,...,2,2,2,2,0,0,0,0,0,0


In [34]:
def get_target(match,number):
    if 'localteamScore' not in match or 'visitorteamScore' not in match:
        raise Exception(f"No data about score in match {match['_id']}")
    local_team_score = match['localteamScore'][number]
    visitor_team_score = match['visitorteamScore'][number]
    if local_team_score == visitor_team_score:
        return 0
    elif local_team_score > visitor_team_score:
        return 1
    else:
        return 2

In [35]:
labelcolumn = []
for i in range(len(df)):
    labelcolumn.append(get_target(df,i))


In [36]:
new_df['winGame'] = labelcolumn

In [37]:
new_df

,attacksLocalteam_10,attacksLocalteam_20,attacksLocalteam_30,attacksLocalteam_40,attacksLocalteam_50,attacksLocalteam_60,possessionLocalteam_10,possessionLocalteam_20,possessionLocalteam_30,possessionLocalteam_40,...,cornersVisitorTeam_40,cornersVisitorTeam_50,cornersVisitorTeam_60,goalsLocalteam_10,goalsLocalteam_20,goalsLocalteam_30,goalsLocalteam_40,goalsLocalteam_50,goalsLocalteam_60,winGame
0,20,39,61,78,101,120,66,68,71,71,...,0,0,0,0,0,0,0,1,1,1
1,16,26,33,47,58,70,59,52,48,46,...,5,5,5,0,0,0,0,0,0,1
2,16,24,38,48,62,78,40,43,44,41,...,2,2,3,1,1,1,1,1,1,2
3,21,38,52,64,80,98,51,54,55,55,...,2,3,3,0,0,1,1,1,1,0
4,15,30,34,46,58,70,61,69,72,70,...,0,0,0,0,0,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76385,18,28,43,51,65,76,62,52,52,51,...,1,1,2,0,0,0,0,0,0,0
76386,15,27,29,34,45,59,34,44,41,40,...,2,4,4,0,0,0,0,0,0,2
76387,19,34,49,65,77,89,75,58,53,56,...,0,0,0,0,0,0,0,0,0,0
76388,6,18,31,45,64,77,24,32,36,38,...,2,2,2,0,0,0,0,0,0,0


## Split label and features

In [50]:
mX = new_df.drop('winGame', axis=1)
vY = new_df['winGame']

## Show mX and vY

In [51]:
mX

,attacksLocalteam_10,attacksLocalteam_20,attacksLocalteam_30,attacksLocalteam_40,attacksLocalteam_50,attacksLocalteam_60,possessionLocalteam_10,possessionLocalteam_20,possessionLocalteam_30,possessionLocalteam_40,...,cornersVisitorTeam_30,cornersVisitorTeam_40,cornersVisitorTeam_50,cornersVisitorTeam_60,goalsLocalteam_10,goalsLocalteam_20,goalsLocalteam_30,goalsLocalteam_40,goalsLocalteam_50,goalsLocalteam_60
0,20,39,61,78,101,120,66,68,71,71,...,0,0,0,0,0,0,0,0,1,1
1,16,26,33,47,58,70,59,52,48,46,...,5,5,5,5,0,0,0,0,0,0
2,16,24,38,48,62,78,40,43,44,41,...,2,2,2,3,1,1,1,1,1,1
3,21,38,52,64,80,98,51,54,55,55,...,2,2,3,3,0,0,1,1,1,1
4,15,30,34,46,58,70,61,69,72,70,...,0,0,0,0,0,0,1,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76385,18,28,43,51,65,76,62,52,52,51,...,0,1,1,2,0,0,0,0,0,0
76386,15,27,29,34,45,59,34,44,41,40,...,2,2,4,4,0,0,0,0,0,0
76387,19,34,49,65,77,89,75,58,53,56,...,0,0,0,0,0,0,0,0,0,0
76388,6,18,31,45,64,77,24,32,36,38,...,2,2,2,2,0,0,0,0,0,0


In [52]:
vY

0        1
1        1
2        2
3        0
4        1
        ..
76385    0
76386    2
76387    0
76388    0
76389    0
Name: winGame, Length: 76390, dtype: int64

## Convert to numpy

In [64]:
target_plot(mX,vY,'./visualization')


In [41]:
mX = mX.to_numpy()  # Convert DataFrame to numpy array
vY = vY.to_numpy()  # Convert Series to numpy array

In [42]:
mX, vY

(array([[20, 39, 61, ...,  0,  1,  1],
        [16, 26, 33, ...,  0,  0,  0],
        [16, 24, 38, ...,  1,  1,  1],
        ...,
        [19, 34, 49, ...,  0,  0,  0],
        [ 6, 18, 31, ...,  0,  0,  0],
        [ 5, 17, 28, ...,  0,  0,  1]], dtype=int64),
 array([1, 1, 2, ..., 0, 0, 0], dtype=int64))

In [67]:
target_plot(mX,vY,'./visualization')

In [27]:
mX_train, mX_test, vY_train, vY_test = train_test_split(mX, vY, test_size=0.2, random_state=42)

In [29]:
svm_model = SVC(C = 0.001, kernel = 'linear')

# Train the SVM model
svm_model.fit(mX_train, vY_train)


SVC(C=0.001, kernel='linear')

In [65]:
# y_pred = svm_model.predict(mX_test)

# # Evaluate the model's accuracy
# accuracy = svm_model.score(vY_test_int, y_pred)
# print("Accuracy:", accuracy)


In [83]:


# Define and train XGBoost classifier
xgb_model = xgb.XGBClassifier()
xgb_model.fit(mX_train, vY_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [84]:
y_pred_xgb = xgb_model.predict(mX_test)


accuracy_xgb = accuracy_score(vY_test, y_pred_xgb)
print("XGBoost Accuracy:", accuracy_xgb)

XGBoost Accuracy: 0.6034166775755989


## Correlation - Check what needs to be thrown away

In [41]:
correlation = new_df.corr()

In [24]:
# client = MongoClient('mongodb://127.0.0.1:27017/')

# # Select the database
# db = client['Project']

# # Select the collection
# collection = db['Data']
# dw = pd.read_csv("data.csv")
# dw

## filter columns
1. discrete columns: \n
    "wether humidity, weather wind - convert column to float \n
    to get information about get_info
    extract spcieal_col:
    match time- convert to to several column: [ month,hour, weekday ]
    
    
    
    
    


In [7]:
# '_id',
#  'weatherCode',
#  'weatherType',
#  'weatherTemperatureUnit',
#  'weatherClouds',

# discrete=[
 
#  'localteamFormation',
#  'visitorteamFormation',
 
#  'htScore', #not in training
#  'ftScore', #not in training 
#  'etScore', #not in training
#  'leagueId',
#  'groupId',
#  'venueId'
# ]

# continious=[
# 'visitorteamPenScore',
#  'localteamPenScore',
#   'visitorteamScore',
#  'localteamScore',
 
#  ,
#  'weatherTemperatureTemp',
#  'weatherWindDegree',
 
 
#  #tbd check the nect columns and filter to irrelvsant discreete or conitnious
#  'refereeId',
#  'localteamId',
#  'visitorteamId',
#  'standingsLocalTeamPosition','standingsVisitorTeamPosition']

# # ,'weatherHumidity','weatherWindSpeed'

# spcieal_col=['matchTime','possessionLocalteam','possessionVisitorTeam','attacksLocalteam','attacksVisitorTeam','dangerousAttacksLocalteam','dangerousAttacksVisitorTeam'
#                 , 'offTargetLocalteam',
#             'offTargetVisitorTeam',
#             'onTargetLocalteam',
#             'onTargetVisitorTeam', 'cornersLocalteam',
#             'cornersVisitorTeam', 'goalsLocalteam',
#             'goalsVisitorTeam','finalStats']

# irrelenvat=['updated','__v','Unnamed: 0','matchId','seasonId',]

#  get_info=['visitorteamPenScore', 'htScore','etScore',
#  'ftScore','finalStats','matchTimeObject','stageId','roundId','aggregateId']

In [8]:
# df['venueId'].value_counts()
# df["weatherWindSpeed"]

In [9]:
# def convert_wind_speed(df):
#     if df['weatherWindSpeed'].dtype == float:
#         return df
        
#     # Remove 'm/s' from the values and convert to float
#     df['weatherWindSpeed'] = df['weatherWindSpeed'].str.rstrip(' m/s').astype(float)
#     return df


In [10]:
# df = convert_wind_speed(df)
# df['weatherWindSpeed']

In [11]:
# df['weatherHumidity']

In [12]:
# def convert_humidity(df):
#     if df['weatherWindSpeed'].dtype == float:
#         return df
#     # Remove '%' from the values, convert to float, and divide by 100
#     df['weatherHumidity'] = df['weatherHumidity'].str.rstrip('%').astype(float) / 100
#     return df

In [13]:
# df = convert_humidity(df)
